# Coursera CapStone Project

In [1]:
import pandas as pd
import numpy as np

In [2]:
print ('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Assignment 1 : Scrapping the neighborhoods in Toronto from Wiki

In [28]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [29]:
df.describe()

,Postal Code,Borough,Neighborhood
count,180,180,103
unique,180,11,98
top,M2S,Not assigned,Downsview
freq,1,77,4


In [30]:
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)

In [32]:
df.describe()

,Postal Code,Borough,Neighborhood
count,103,103,103
unique,103,10,98
top,M1W,North York,Downsview
freq,1,24,4


RangeIndex(start=0, stop=180, step=1)